In [2]:
import random
from enum import StrEnum
from threading import ExceptHookArgs
from typing import Self
from rich import print as rprint


class Names(StrEnum):
    JOHN = "John Lennon"
    PAUL = "Paul McCartney"
    GEORGE = "George Harrison"
    RINGO = "Ringo Starr"


class Beetle:
    health_points: int
    name: Names
    max_health_points: int
    max_damage: int

    def __init__(
        self,
        health_points: int = 100,
        name: Names = Names.JOHN,
        max_health_points: int = 100,
        max_damage: int = 20,
    ) -> None:
        self.health_points = health_points
        self.name = name
        self.max_health_points = max_health_points
        self.max_damage = max_damage

    def __eq__(self, other: Self) -> bool:
        return self.health_points == other.health_points

    def __lt__(self, other: Self) -> bool:
        return self.health_points < other.health_points

    def __le__(self, other: Self) -> bool:
        return self.health_points <= other.health_points

    def __str__(self) -> str:
        return f'Beetle(name="{self.name}", hp={self.health_points!r})'

    def styling(self) -> str:
        if self.name is Names.JOHN:
            return "в стиле Джонни"
        elif self.name is Names.PAUL:
            return "в стиле Маккартни"
        return "без стиля"

    def is_alive(self) -> bool:
        return self.health_points > 0

    def heal(self, amount: int) -> None:
        old_hp = self.health_points
        self.health_points = min(self.health_points + amount, self.max_health_points)
        healed = self.health_points - old_hp
        if healed > 0:
            rprint(f"  [bold green]{self.name} лечится +{healed} HP (теперь {self.health_points}/{self.max_health_points} HP)[/bold green]")

    def attack(self, other: Self) -> None:
        damage = random.randint(1, self.max_damage)
        rprint(f"  [bold cyan]{self.name}[/bold cyan] атакует [bold red]{other.name}[/bold red] {self.styling()} с уроном [bold yellow]{damage}[/bold yellow]")
        other.health_points -= damage
        
        if not other.is_alive():
            rprint(f"  [bold red]{other.name} побеждён![/bold red]")
            self.heal(10)
        else:
            rprint(f"  [dim]У {other.name} осталось {other.health_points}/{other.max_health_points} HP[/dim]")


class BeetlesArmy:
    beetles_list: list[Beetle]
    beetles_name: Names
    beetles_max_health_points: int
    beetles_max_damage: int

    def __init__(
        self,
        beetles_name: Names,
        beetles_army_size: int = 20,
        beetles_max_health_points: int = 100,
        beetles_max_damage: int = 20,
    ):
        self.beetles_list = []
        self.beetles_name = beetles_name
        self.beetles_max_health_points = beetles_max_health_points
        self.beetles_max_damage = beetles_max_damage

        for _ in range(beetles_army_size):
            hp = random.randint(1, self.beetles_max_health_points)
            beetle = Beetle(
                health_points=hp,
                name=self.beetles_name,
                max_health_points=self.beetles_max_health_points,
                max_damage=self.beetles_max_damage,
            )
            self.beetles_list.append(beetle)

    def __len__(self) -> int:
        return len([b for b in self.beetles_list if b.is_alive()])

    def __eq__(self, other: Self) -> bool:
        return len(self) == len(other)

    def __lt__(self, other: Self) -> bool:
        return len(self) < len(other)

    def __le__(self, other: Self) -> bool:
        return len(self) <= len(other)

    def __add__(self, other: Self) -> Self:
        if self.beetles_name != other.beetles_name:
            raise ValueError("Нельзя объединить армии с разными именами")
        new_beetles_list: list[Beetle] = self.beetles_list + other.beetles_list
        new_army = self.__class__(
            beetles_army_size=1,
            beetles_name=self.beetles_name,
            beetles_max_health_points=self.beetles_max_health_points,
            beetles_max_damage=self.beetles_max_damage,
        )
        new_army.beetles_list = new_beetles_list
        return new_army

    def get_alive_beetles(self) -> list[Beetle]:
        return [b for b in self.beetles_list if b.is_alive()]

    def print_army_listing(self):
        for beetle in self.beetles_list:
            if beetle.is_alive():
                rprint(f"  [green]{beetle}[/green]")
            else:
                rprint(f"  [dim red]{beetle} (МЁРТВ)[/dim red]")

    def battle(self, other: Self) -> None:
        rprint(f"\n[bold magenta]{'='*70}[/bold magenta]")
        rprint(f"[bold magenta]БИТВА НАЧИНАЕТСЯ: {self.beetles_name} VS {other.beetles_name}[/bold magenta]")
        rprint(f"[bold magenta]{'='*70}[/bold magenta]\n")

        round_num = 1
        while len(self) > 0 and len(other) > 0:
            rprint(f"\n[bold blue]Раунд {round_num}[/bold blue]")
            rprint(f"[cyan]{self.beetles_name}: {len(self)} жуков живы[/cyan]")
            rprint(f"[cyan]{other.beetles_name}: {len(other)} жуков живы[/cyan]\n")

            attacker = self.get_alive_beetles()[0]
            defender = other.get_alive_beetles()[0]
            attacker.attack(defender)

            if len(other) == 0:
                break

            rprint()

            attacker = other.get_alive_beetles()[0]
            defender = self.get_alive_beetles()[0]
            attacker.attack(defender)

            round_num += 1

        rprint(f"\n[bold magenta]{'='*70}[/bold magenta]")
        if len(self) > 0:
            rprint(f"[bold green]{self.beetles_name} ПОБЕДИЛИ! (осталось {len(self)} жуков)[/bold green]")
        else:
            rprint(f"[bold green]{other.beetles_name} ПОБЕДИЛИ! (осталось {len(other)} жуков)[/bold green]")
        rprint(f"[bold magenta]{'='*70}[/bold magenta]\n")


if __name__ == "__main__":
    rprint("[bold yellow]Добро пожаловать в Битву Армий Жуков![/bold yellow]\n")

    try:
        num_armies = int(input("Сколько армий создать? (рекомендуется 2): "))
        if num_armies < 2:
            rprint("[red]Нужно минимум 2 армии для битвы![/red]")
            num_armies = 2
    except ValueError:
        rprint("[red]Неверный ввод, используется 2 армии[/red]")
        num_armies = 2

    armies = []
    names_list = list(Names)

    for i in range(num_armies):
        rprint(f"\n[bold cyan]Настройка армии {i+1}[/bold cyan]")
        rprint("Доступные имена:")
        for idx, name in enumerate(names_list):
            rprint(f"  {idx+1}. {name}")

        try:
            name_choice = int(input(f"Выберите имя (1-{len(names_list)}): "))
            if 1 <= name_choice <= len(names_list):
                chosen_name = names_list[name_choice - 1]
            else:
                rprint(f"[yellow]Неверный выбор, используется {names_list[i % len(names_list)]}[/yellow]")
                chosen_name = names_list[i % len(names_list)]
        except ValueError:
            rprint(f"[yellow]Неверный ввод, используется {names_list[i % len(names_list)]}[/yellow]")
            chosen_name = names_list[i % len(names_list)]

        try:
            army_size = int(input("Размер армии: "))
        except ValueError:
            rprint("[yellow]Неверный ввод, используется 10[/yellow]")
            army_size = 10

        try:
            max_hp = int(input("Максимальное здоровье: "))
        except ValueError:
            rprint("[yellow]Неверный ввод, используется 100[/yellow]")
            max_hp = 100

        try:
            max_damage = int(input("Максимальный урон: "))
        except ValueError:
            rprint("[yellow]Неверный ввод, используется 20[/yellow]")
            max_damage = 20

        army = BeetlesArmy(
            beetles_name=chosen_name,
            beetles_army_size=army_size,
            beetles_max_health_points=max_hp,
            beetles_max_damage=max_damage,
        )
        armies.append(army)

    rprint("\n[bold yellow]Созданные армии:[/bold yellow]\n")
    for i, army in enumerate(armies):
        rprint(f"[bold cyan]АРМИЯ {i+1}: {army.beetles_name} ({len(army)} жуков)[/bold cyan]")
        army.print_army_listing()
        rprint()

    if len(armies) >= 2:
        rprint("[bold yellow]Сравнение армий:[/bold yellow]\n")
        for i in range(len(armies)):
            for j in range(i+1, len(armies)):
                a1, a2 = armies[i], armies[j]
                rprint(f"[cyan]{a1.beetles_name}[/cyan] ({len(a1)} жуков) vs [cyan]{a2.beetles_name}[/cyan] ({len(a2)} жуков)")
                if a1 == a2:
                    rprint("  [yellow]Армии РАВНЫ[/yellow]")
                elif a1 > a2:
                    rprint(f"  [green]{a1.beetles_name} СИЛЬНЕЕ[/green]")
                else:
                    rprint(f"  [green]{a2.beetles_name} СИЛЬНЕЕ[/green]")
        rprint()

    if len(armies) >= 2:
        armies[0].battle(armies[1])


Добро пожаловать в Битву Армий Жуков!

Настройка армии 1

Доступные имена:

1. John Lennon

2. Paul McCartney

...

Paul McCartney побеждён!

Ringo Starr лечится +9 HP (теперь 10/10 HP)

======================================================================

Ringo Starr ПОБЕДИЛИ! (осталось 3 жуков)

======================================================================